In [1]:
import openmm
from openmm import unit
from openmm import app
import numpy as np

from sys import stdout
import mdtraj as md
#from openmmtools import integrators

In [2]:
ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
pdb = app.PDBFile('cis_pro.pdb')

system = ff.createSystem(
    topology=pdb.getTopology(), 
    removeCMMotion=True,
    nonbondedMethod=app.NoCutoff,
    constraints=app.HBonds, 
    rigidWater=True
    )

In [ ]:
## creates openmm system for passing to bg to calculate energies

# xml = openmm.XmlSerializer.serialize(system)
# xml_file = open('cis_pro_xmlsystem.txt','w')
# xml_file.write(xml)
# xml_file.close()

In [3]:
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 2.0 * unit.femtosecond
reportInterval = 2000
steps = 5E+8
fname = 'cis_pro_300K_long'
#time = (steps*timestep).value_in_unit(unit.nanosecond)
parametersdict = {'Collision rate':collision_rate,'Temperature':temperature,'Timestep':timestep,'Report Interval':reportInterval}
import pickle
f_p = open(f'parameters{fname}.pkl','wb')
pickle.dump(parametersdict,f_p)
f_p.close

<function BufferedWriter.close>

In [ ]:
integrator = openmm.LangevinIntegrator(temperature,collision_rate,timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
platform = openmm.Platform.getPlatform(2)

positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

simulation = app.Simulation(pdb.getTopology(), system, integrator,platform,platformProperties=properties_dict)
#print(platform.getPropertyValue(simulation.context,property='Precision'))
#print(simulation.context.getPlatform())
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

simulation.reporters.append(app.StateDataReporter(stdout, reportInterval, step=True, potentialEnergy=True,temperature=True))
simulation.reporters.append(app.DCDReporter(f'{fname}.dcd',reportInterval))
#h5_reporter = reporters.HDF5Reporter('output.h5',reportInterval)
#simulation.reporters.append(h5_reporter)

In [ ]:
simulation.step(steps)
#h5_reporter.close()

In [ ]:
import mdtraj as md

#traj = md.load('pro_trajectory.dcd',top=md.Topology.from_openmm(topology))
traj = md.load('cis_pro_300K.dcd',top='cis_pro.pdb')
import nglview
view = nglview.show_mdtraj(traj)
view